# Way 1 : From The ExtractedFeatures Folder

In [2]:
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat
import scipy.io as sio
from sklearn.utils import shuffle 
import matplotlib.pyplot as plt


In [2]:
# Path to the folder containing the mat files
mat_folder = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/ExtractedFeatures"

# Initialize arrays to store data sizes and maximum size
sizes = []
max_vals = 0

# Loop through all files in the folder
for filename in os.listdir(mat_folder):
    if filename.endswith(".mat"):
        mat_file = os.path.join(mat_folder, filename)
        loaded_data = loadmat(mat_file)
        # Loop through the variables in the loaded file
        for var_name in loaded_data:
            # Check if the variable name starts with 'de_LDS'
            if var_name.startswith('de_LDS') and isinstance(loaded_data[var_name], np.ndarray) and len(loaded_data[var_name].shape) == 3:
                sizes.append(loaded_data[var_name].shape[1])
                max_vals = max(max_vals, loaded_data[var_name].shape[1])

print(max_vals)

265


In [3]:


# Calculate the total number of trials
total_trials = 30

# Initialize the data array
data = np.zeros((total_trials, 62, max_vals, 5))

# Loop through all files and variables again to load data
index = 0
for filename in os.listdir(mat_folder):
    if filename.endswith(".mat"):
        mat_file = os.path.join(mat_folder, filename)
        loaded_data = loadmat(mat_file)
        
        for var_name in loaded_data:
            # Check if the variable name starts with 'de_LDS'
            if var_name.startswith('de_LDS') and isinstance(loaded_data[var_name], np.ndarray) and len(loaded_data[var_name].shape) == 3:
                num_channels = loaded_data[var_name].shape[0]
                num_elements = loaded_data[var_name].shape[1]
                data[index, :num_channels, :num_elements, :] = loaded_data[var_name]
                index += 1

# Print the shape of the data array
print("Shape of data array:", data.shape)


Shape of data array: (30, 62, 265, 5)


# Way 2: From the Preprocessed eeg

https://www.programmersought.com/article/336211155520/

In [7]:
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import numpy as np
import math
import os

def compute_DE(signal):
    variance = np.var(signal, ddof=1)
    return math.log(2 * math.pi * math.e * variance) / 2

def load_data():
    data_dir = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/Preprocessed_EEG/"
    fs = 200
    fStart = [1, 4, 8, 14, 31]
    fEnd = [3, 7, 13, 30, 50]
    all_channels = range(62)

    filename_label = "label"
    label = loadmat(data_dir + filename_label)
    label = label["label"][0]

    datasets_X, datasets_y = [], []

    for filename_data in os.listdir(data_dir):
        if filename_data in ["label.mat", "readme.txt"]:
            continue

        data_all = loadmat(data_dir + filename_data)
        scenes = list(data_all.keys())[3:]

        for index, scene in enumerate(scenes):
            dataset_X = []
            data = data_all[scene][:,]  # Considering all channels

            for band_index, band in enumerate(fStart):
                b, a = signal.butter(4, [fStart[band_index]/fs, fEnd[band_index]/fs], 'bandpass')
                filtedData = signal.filtfilt(b, a, data)
                filtedData_de = []

                for lead in all_channels:
                    filtedData_split = []
                    for de_index in range(0, filtedData.shape[1] - fs, fs):
                        filtedData_split.append(compute_DE(filtedData[lead, de_index: de_index + fs]))
                    if len(filtedData_split) < 265:  # Max point size is 53001 so, 53001/200 = 265
                        filtedData_split += [0] * (265-len(filtedData_split))
                    filtedData_de.append(filtedData_split)
                
                filtedData_de = np.array(filtedData_de)
                dataset_X.append(filtedData_de)

            datasets_X.append(dataset_X)
            datasets_y.append(label[index])

    datasets_X, datasets_y = np.array(datasets_X), np.array(datasets_y)

    return datasets_X, datasets_y


datasets_X, datasets_y = load_data()
print(datasets_X.shape)
print(datasets_y.shape)


(30, 5, 62, 265)
(30,)


In [8]:
new_datasets_X = np.copy(datasets_X)
new_datasets_X = np.transpose(new_datasets_X, (0, 2, 3, 1))
print(new_datasets_X.shape)

(30, 62, 265, 5)


In [69]:
np.save('SEED_DE_X.npy', new_datasets_X)
np.save('SEED_DE_y.npy', datasets_y)


In [3]:
X  = np.load("E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/DE_SEED_X_y/SEED_DE_X.npy")
y  = np.load("E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/DE_SEED_X_y/SEED_DE_y.npy")

# Explainations

In [ ]:
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import numpy as np
import math
import os

file_path = 'E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/Preprocessed_EEG/1_20131027.mat'
fs = 200
fStart = [1, 4, 8, 14, 31]
fEnd = [3, 7, 13, 30, 50]
channel = [3, 7, 13, 23]

# Assuming data is defined before the loop
# data = ...
data_all = loadmat(file_path)
data = data_all['djc_eeg2'][channel]

for band_index, band in enumerate(fStart):
    b, a = signal.butter(4, [fStart[band_index]/fs, fEnd[band_index]/fs], 'bandpass')
    filtedData = signal.filtfilt(b, a, data)
    
    plt.figure(figsize=(12, 8))
    
    for lead in range(filtedData.shape[0]):
        plt.subplot(filtedData.shape[0], 1, lead+1)
        plt.plot(filtedData[lead, :], label=f'Channel {lead+1}')
        plt.title(f'Filtered Data - Band {band_index} - Channel {lead+1}')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.legend()

    plt.tight_layout()
    plt.show()
